In [6]:
import pandas as pd
import numpy as np

uber_trips_2014 = pd.read_csv('Datathon Materials/uber_trips_2014.csv')
uber_trips_2014.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base
0,4/1/14 0:11,40.7690,-73.9549,B02512
1,4/1/14 0:17,40.7267,-74.0345,B02512
2,4/1/14 0:21,40.7316,-73.9873,B02512
3,4/1/14 0:28,40.7588,-73.9776,B02512
4,4/1/14 0:33,40.7594,-73.9722,B02512


In [3]:
uber_trips_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4534327 entries, 0 to 4534326
Data columns (total 4 columns):
pickup_datetime     object
pickup_latitude     float64
pickup_longitude    float64
base                object
dtypes: float64(2), object(2)
memory usage: 138.4+ MB


In [4]:
df_locations = pd.read_csv('Datathon Materials/geographic.csv')
df_locations.head()

,BK88,QN52,QN48,QN51,QN27,BX35,BX98,QN07,MN06,QN02,...,MN25,MN24,MN23,MN13,MN15,MN32,MN33,MN99,QN18,QN29
0,-73.976051,-73.794934,-73.775740,-73.803792,-73.861099,-73.896967,-73.872873,-73.757258,-73.946080,-73.760316,...,-74.000784,-73.992605,-73.989904,-74.001535,-73.993833,-73.938048,-73.935054,-74.010930,-73.858726,-73.867911
1,40.631284,40.757806,40.743334,40.775618,40.763673,40.834350,40.785983,40.718147,40.821271,40.675118,...,40.694295,40.724145,40.734443,40.762653,40.772940,40.780838,40.791695,40.684500,40.735717,40.744942
2,-73.977167,-73.794749,-73.775799,-73.800991,-73.859931,-73.896787,-73.872886,-73.755890,-73.946404,-73.758806,...,-74.000960,-73.992634,-73.989870,-74.001339,-73.993789,-73.937903,-73.935108,-74.011934,-73.858672,-73.867877
3,40.630755,40.755636,40.743326,40.775397,40.762209,40.834133,40.785971,40.716307,40.820580,40.673014,...,40.694077,40.724065,40.733535,40.762217,40.772912,40.780758,40.791658,40.683896,40.735627,40.744824
4,-73.977000,-73.794579,-73.778060,-73.798653,-73.859668,-73.896303,-73.872983,-73.754702,-73.946720,-73.758598,...,-74.003015,-73.993093,-73.990028,-74.001279,-73.993727,-73.937796,-73.935173,-74.012177,-73.858585,-73.867852


In [7]:
from shapely.geometry import Point
from shapely.geometry import MultiPoint

polys = []
polys_nta = []
for column in df_locations.columns.values:
    xy_points = df_locations[column]
    x = np.array(xy_points)
    #x = x[np.logical_not(np.isnan(x))]
    y=[]
    for i in range(0,int((x.shape[0])/2),2):
        a=x[i]
        b=x[i+1]
        y.append([a,b])
        
    
    polys.append(np.nanmean(y,axis=0)) 
    polys_nta.append(column)
polys

[array([-73.98789731,  40.63066724]),
 array([-73.8071623 ,  40.75303473]),
 array([-73.79014061,  40.750217  ]),
 array([-73.80767235,  40.76810195]),
 array([-73.86587071,  40.76338293]),
 array([-73.90503689,  40.82562415]),
 array([-73.88070609,  40.7935005 ]),
 array([-73.76406789,  40.71050489]),
 array([-73.95607448,  40.81967302]),
 array([-73.77251739,  40.67298008]),
 array([-73.76788636,  40.76290174]),
 array([-73.96471679,  40.59838252]),
 array([-73.84493708,  40.83436036]),
 array([-73.78552758,  40.73499121]),
 array([-73.98288058,  40.75644593]),
 array([-73.9695875 ,  40.74841203]),
 array([-73.78012472,  40.78786533]),
 array([-73.76423117,  40.69280015]),
 array([-73.82923669,  40.69593568]),
 array([-73.85669659,  40.74084225]),
 array([-73.81708135,  40.67648725]),
 array([-73.77864589,  40.71904442]),
 array([-73.94637355,  40.60553573]),
 array([-73.97751903,  40.65533071]),
 array([-73.97641428,  40.64072705]),
 array([-73.95203147,  40.64555597]),
 array([-73.

In [8]:
from scipy.spatial import KDTree
kd = KDTree(polys)

def lonlat2NTA(x):
    return polys_nta[kd.query(x)[1]]


uber_trips_2014['nta_code'] = uber_trips_2014[['pickup_longitude','pickup_latitude']].apply(lonlat2NTA,axis=1)
uber_trips_2014.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base,nta_code
0,4/1/14 0:11,40.7690,-73.9549,B02512,MN40
1,4/1/14 0:17,40.7267,-74.0345,B02512,MN24
2,4/1/14 0:21,40.7316,-73.9873,B02512,MN22
3,4/1/14 0:28,40.7588,-73.9776,B02512,MN17
4,4/1/14 0:33,40.7594,-73.9722,B02512,MN17


In [9]:
from datetime import datetime
def convert_datetime_date(x):
    [date,time] = x.split(' ')
    return date
def convert_datetime_time(x):
    [date,time] = x.split(' ')
    try:
        [hh,mm] = time.split(':')
    except ValueError:
        [hh,mm,ss] = time.split(':')
    return int(hh)


uber_trips_2014['date'] = uber_trips_2014['pickup_datetime'].apply(convert_datetime_date)
uber_trips_2014['hour'] = uber_trips_2014['pickup_datetime'].apply(convert_datetime_time)

uber_trips_2014['demand'] = 1

uber_trips_2014.head()

,pickup_datetime,pickup_latitude,pickup_longitude,base,nta_code,date,hour,demand
0,4/1/14 0:11,40.7690,-73.9549,B02512,MN40,4/1/14,0,1
1,4/1/14 0:17,40.7267,-74.0345,B02512,MN24,4/1/14,0,1
2,4/1/14 0:21,40.7316,-73.9873,B02512,MN22,4/1/14,0,1
3,4/1/14 0:28,40.7588,-73.9776,B02512,MN17,4/1/14,0,1
4,4/1/14 0:33,40.7594,-73.9722,B02512,MN17,4/1/14,0,1


In [10]:
dataset_2014 = uber_trips_2014.groupby(['date','hour','nta_code'], as_index=False)['demand'].sum()
def convert_datetime_isweekday(x):
    try:
        datetimeobj = datetime.strptime(x, "%m/%d/%y")
    except ValueError:
        datetimeobj = datetime.strptime(x, "%m/%d/%Y")
    weekday = datetimeobj.isoweekday()
    if weekday<=5:
        isweekday = 1
    else:
        isweekday = 0
    return isweekday

dataset_2014['isweekday'] = dataset_2014['date'].apply(convert_datetime_isweekday)
dataset_2014.head()

,date,hour,nta_code,demand,isweekday
0,4/1/14,0,BK34,1,1
1,4/1/14,0,BK37,1,1
2,4/1/14,0,BK60,1,1
3,4/1/14,0,BK64,1,1
4,4/1/14,0,BK69,1,1


In [12]:
dataset_2014.to_csv('uber2014.csv')